# Chargement 
### Librairies

In [ ]:
import numpy as np
import pandas as pd
import math
from math import sqrt
import random
from pandas import read_csv
from scipy.spatial.distance import euclidean

### Jeu de données

In [ ]:
got = pd.read_csv('https://raw.githubusercontent.com/astridgcn/Info/main/got.csv', header = 0, index_col = 0, sep= ';', decimal = '.')
got

,name,male,culture,house,book1,book2,book3,book4,book5,isMarried,isNoble,numDeadRelations,popularity,isAlive
S.No,,,,,,,,,,,,,,
1,Arya Stark,0,Northmen,House Stark,1,1,1,1,1,0,1,8,1.0,1
2,Barristan Selmy,1,Westeros,House Selmy,1,1,1,1,1,0,1,0,1.0,1
3,Bran Stark,1,Northmen,House Stark,1,1,1,1,1,0,1,7,1.0,1
4,Cersei Lannister,0,Westerlands,House Lannister,1,1,1,1,1,1,1,9,1.0,1
5,Daenerys Targaryen,0,Valyrian,House Targaryen,1,1,1,1,1,1,1,15,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Walder Frey (son of Merrett),1,NaN,House Frey,0,0,0,0,0,0,0,0,0.0,0
250,Walder Frey (son of Ryman),1,Rivermen,House Frey,0,0,0,0,0,0,0,0,0.0,1
261,Wate,0,NaN,House Targaryen,0,0,0,0,0,0,0,0,0.0,1


In [ ]:
print(got.shape) # forme des données
print(got.dtypes) # types de données

(1937, 14)
name                 object
male                  int64
culture              object
house                object
book1                 int64
book2                 int64
book3                 int64
book4                 int64
book5                 int64
isMarried             int64
isNoble               int64
numDeadRelations      int64
popularity          float64
isAlive               int64
dtype: object


In [ ]:
got.iloc[1,3] # test index

'House Selmy'

## Traitement

### Suppression des données manquantes

In [ ]:
#got.dropna()

### Sélection des données
#### Étude des corrélations

In [ ]:
corr = got.corr(method = 'pearson')
corr
# Forte corr : male, book1, book4, numDeadRelations, popularity

,male,book1,book2,book3,book4,book5,isMarried,isNoble,numDeadRelations,popularity,isAlive
male,1.000000,0.084993,0.061676,0.024528,-0.061519,0.063076,-0.082852,0.157680,0.029365,0.131311,-0.143593
book1,0.084993,1.000000,0.390124,0.241641,0.077017,0.262576,0.084279,0.092913,0.201701,0.346472,-0.146926
book2,0.061676,0.390124,1.000000,0.448549,0.173101,0.223518,0.156528,0.020236,0.086390,0.273509,-0.068381
book3,0.024528,0.241641,0.448549,1.000000,0.370511,0.281976,0.148731,0.020408,0.063746,0.241490,0.006675
book4,-0.061519,0.077017,0.173101,0.370511,1.000000,0.334487,0.125052,0.075244,0.004823,0.142138,0.269860
book5,0.063076,0.262576,0.223518,0.281976,0.334487,1.000000,0.092848,0.098406,0.114055,0.239003,0.034009
isMarried,-0.082852,0.084279,0.156528,0.148731,0.125052,0.092848,1.000000,0.162069,0.198547,0.294394,-0.044258
isNoble,0.157680,0.092913,0.020236,0.020408,0.075244,0.098406,0.162069,1.000000,0.094722,0.174041,-0.042388
numDeadRelations,0.029365,0.201701,0.086390,0.063746,0.004823,0.114055,0.198547,0.094722,1.000000,0.585918,-0.176926
popularity,0.131311,0.346472,0.273509,0.241490,0.142138,0.239003,0.294394,0.174041,0.585918,1.000000,-0.169552


## Distances
### Entre deux personnes
#### Utilisation de la distance de Manhattan (pour les vecteurs)

In [ ]:
def distance(prenom1, prenom2):
  personne1 = got.loc[got['name'] == prenom1,('male', 'book1', 'book4', 'numDeadRelations', 'popularity')] # on cherche le 1er prénom
  personne1 = personne1[['male', 'book1', 'book4', 'numDeadRelations', 'popularity']].values # on récupère les variables qui nous intéressent
  personne2 = got.loc[got['name'] == prenom2,('male', 'book1', 'book4', 'numDeadRelations', 'popularity')] # on cherche le 2ème prénom
  personne2 = personne2[['male', 'book1', 'book4', 'numDeadRelations', 'popularity']].values # on récupère les variables qui nous intéressent

  return np.sqrt(np.sum((personne1 - personne2) ** 2)) # pour ne pas utiliser SciPy

#### Test avec exemple

In [ ]:
distance('Arya Stark', 'Tyrion Lannister')

4.123105625617661

### Entre une personne et les autres

In [ ]:
def distanceall(prenom):
  #distances = [] # liste des distances
  distances = pd.DataFrame(columns = ['Prénom', 'Distance']) # on crée un tableau
  for i in range(len(got)):
    if got.iloc[i,0] != prenom : # on exclut la personne
      prenom2 = got.iloc[i,0] # on récupère le 2ème prénom
      dist = distance(prenom, prenom2) # on calcule la distance entre les deux 
      ligne = [prenom2, dist]
      distances.loc[len(distances)] = ligne # on l'ajoute à notre tableau

  distances.sort_values(by = ['Distance'], inplace = True) # on range le tableau par ordre croissant

  return distances # on affiche la liste 

#### Test avec exemple

In [ ]:
distanceall('Arya Stark')

,Prénom,Distance
2,Cersei Lannister,1.000000
41,Edmure Tully,1.057671
56,Rickon Stark,1.095832
74,Elia Martell,1.145406
110,Rhaella Targaryen,1.208696
...,...,...
1851,Aegon Frey (son of Aenys),8.246211
1887,Green Gergen,8.246211
1900,Lothar,8.246211
16,Jorah Mormont,10.007703


## Prédiction
### Fonction de prédiction

In [ ]:
def predict (prenom, k):
  distances = distanceall(prenom) # on calcule sa distance avec tous les autres
  distancesk = distances.head(k) # on affiche les k premiers
  kAlive = 0
  # On récupère les valeurs 'isAlive' des k plus proches voisins
  for i in range(k):
    prenomn = distancesk.iloc[i,0] # on récupère le prénom du k plus proche voisin
    personne = got.loc[got['name'] == prenomn, ('isAlive')].values
    kAlive += personne[0] # on augmente le compteur de voisin·es vivant·es
  mid = math.ceil(k/2)
  
  # On récupère la vraie valeur de isAlive
  alive = got.loc[got['name'] == prenom,'isAlive'] # on cherche le 1er prénom
  # On classifie selon les valeurs 'isAlive' des k plus proches voisins
  print(kAlive, "de ses", k, "plus proches voisins sont vivants.")
  if kAlive >= mid:
    print("Donc", prenom, "est probablement vivant·e.")
    if alive.values[0] == 1:
      print("Ce qui est une bonne prédiction")
    else:
      print("Ce qui est une mauvaise prédiction")
  else:
    print("Donc", prenom, "est probablement mort·e.")
    if alive.values[0] == 0:
      print("Ce qui est une bonne prédiction")
    else:
      print("Ce qui est une mauvaise prédiction")

#### Test avec exemple

In [ ]:
predict('Arya Stark', 5)

3 de ses 5 plus proches voisins sont vivants.
Donc Arya Stark est probablement vivant·e.
Ce qui est une bonne prédiction


### Leave-one-out
- Tester plusieurs pondérations
- Tester pour chaque exemple 
- Mesurer taux de prédictions correctes (nb succès / nb exemple total)

#### Entre deux personnes
Avec les poids

In [ ]:
def distancep(prenom1, prenom2, poids1, poids2, poids3, poids4, poids5):
  personne1 = got.loc[got['name'] == 'Arya Stark',('male', 'book1', 'book4', 'numDeadRelations', 'popularity')] # on cherche le 1er prénom
  personne1 = personne1[['male', 'book1', 'book4', 'numDeadRelations', 'popularity']].values # on récupère les variables qui nous intéressent
  personne2 = got.loc[got['name'] == 'Tyrion Lannister',('male', 'book1', 'book4', 'numDeadRelations', 'popularity')] # on cherche le 2ème prénom
  personne2 = personne2[['male', 'book1', 'book4', 'numDeadRelations', 'popularity']].values # on récupère les variables qui nous intéressent

  dist = sqrt((personne2[0, 0] * poids1 - personne1[0, 0] * poids1)**2 + (personne2[0, 1] * poids2 - personne1[0, 1] * poids2)**2 + (personne2[0, 2] * poids3 - personne1[0, 2] * poids3)**2 + (personne2[0, 3] * poids4 - personne1[0, 3] * poids4)**2 + (personne2[0, 4] * poids5 - personne1[0, 4] * poids5)**2)
  return dist # on calcule la distance pondérée

Test avec deux exemples

In [ ]:
distancep('Arya Stark', 'Tyrion Lannister', 0.2, 0.1, 0.1, 0.3, 0.3)

1.2165525060596436

#### Entre une personne et les autres
Avec les poids

In [ ]:
def distanceallp(prenom, poids1, poids2, poids3, poids4, poids5):
  #distances = [] # liste des distances
  distances = pd.DataFrame(columns = ['Prénom', 'Distance']) # on crée un tableau

  # On parcourt tout le tableau
  for i in range(len(got)):
    if got.iloc[i,0] != prenom: # on exclut la personne
      prenom2 = got.iloc[i,0] # on récupère le 2ème prénom
      dist = distancep(prenom, prenom2, poids1, poids2, poids3, poids4, poids5) # on calcule la distance entre les deux 
      ligne = [prenom2, dist]
      distances.loc[len(distances)] = ligne # on l'ajoute à notre tableau

  # On classe les distances
  distances.sort_values(by = ['Distance'], inplace = True) # on range le tableau par ordre croissant
  return distances # on affiche la liste 

#### Essai sur tous

In [23]:
def predictp(k):
  suc = 0 # nombres de bonnes prédictions

  for i in range(len(got)): 
  #for i in range(10):  # test
    prenom = got.iloc[i,0] # on récupère le prenom
    poids1 = round(random.uniform(0, 1), 2) # on génère le 1er poids entre 0 et 1
    poids2 = round(random.uniform(0, 1-poids1), 2) # on génère le 2ème poids entre 0 etle 1er
    poids3 = round(random.uniform(0, 1-(poids2 + poids1)), 2) # on génère le 3ème poids entre 0 et les 2 autres
    poids4 = round(random.uniform(0, 1-(poids3 + poids2 + poids1)), 2) # on génère le 4ème poids entre 0 et les 3 autres 
    poids5 = round(random.uniform(0, 1-(poids4 + poids3 + poids2 + poids1)), 2) # on génère le 5ème poids entre 0 et les 4 autres
    distances = distanceallp(prenom, poids1, poids2, poids3, poids4, poids5) # on calcule sa distance avec tous les autres
    distancesk = distances.head(k) # on affiche les k premiers
    kAlive = 0

    # On récupère la vraie valeur de isAlive
    alive = got.loc[got['name'] == prenom,'isAlive'] # on cherche le 1er prénom

    # On récupère les valeurs 'isAlive' des k plus proches voisins
    for j in range(k):
      prenomn = distancesk.iloc[j,0] # on récupère le prénom du k plus proche voisin
      personne = got.loc[got['name'] == prenomn, ('isAlive')].values
      kAlive += personne[0]
    mid = math.ceil(k/2)
    
    # On classifie selon les valeurs 'isAlive' des k plus proches voisins
    if kAlive >= mid:
      if alive.values[0] == 1:
        suc += 1
    else:
      if alive.values[0] == 0:
        suc += 1
  acc = suc / len(got)
  #acc = suc / 10 # test
  print("Avec", k, "voisins et les poids suivants :", poids1,"pour la variable 'male'", poids2, "pour la variable 'book1'", poids3, "pour la variable 'book4'", poids4, "pour la variable 'numDeadRelations'", poids5, "pour la variable 'popularity', on obtient une précision de", acc)
  return poids1, poids2, poids3, poids4, poids4, k, acc

In [24]:
predictp(5)

Avec 5 voisins et les poids suivants : 0.49 pour la variable 'male' 0.43 pour la variable 'book1' 0.07 pour la variable 'book4' 0.01 pour la variable 'numDeadRelations' 0.0 pour la variable 'popularity', on obtient une précision de 0.9


(0.49, 0.43, 0.07, 0.01, 0.01, 5, 0.9)

## Export / Import
Les paramètres vont maintenant être lus non plus dans votre code mais dans un fichier externe.
Cela vous permettra de réaliser plusieurs simulations en allant à chaque fois lire les paramètres de la simulation dans ce fichier.
Vous ferez également un fichier resultats.txt, dans lequel vous irez écrire les paramètres et le taux de succès de votre algorithme à chaque simulation. 
Ce fichier de paramètres peut être généré automatiquement par un autre script, ou par Excel.

### Export 
#### Test de plusieurs paramètres

In [25]:
def predictpk(n, k):
  for i in range(n): # on le fait n fois (change les pondérations à chaque fois)
    predictp(k)
    print("Avec", k, "voisins et les poids suivants:", poids1,"pour la variable 'male'", poids2, "pour la variable 'book1'", poids3, "pour la variable 'book4'", poids4, "pour la variable 'numDeadRelations'", poids5, "pour la variable 'popularity'", "on obtient une précision de", acc)
  
    filename = "resultats_essai" + str(n) + ".txt"
    line = "Avec " + str(k) + " voisins et les poids suivants : " + str(poids1) + " pour la variable 'male' " + str(poids2) + " pour la variable 'book1' " + str(poids3) + " pour la variable 'book4' " + str(poids4) + " pour la variable 'numDeadRelations' " + str(poids5) + " pour la variable 'popularity', on obtient une précision de " + str(acc)
    with openopen(filename, "w") as f:
      f.write(line)

Test

In [ ]:
predictpk(1,5)

### Import

In [26]:
param = pd.read_csv('https://raw.githubusercontent.com/astridgcn/Info/main/param.csv', header = None, sep= ';')

In [27]:
def predictpimp(k, poids1, poids2, poids3, poids4, poids5):
  suc = 0 # nombres de bonnes prédictions

  for i in range(len(got)):
  #for i in range(10):
    prenom = got.iloc[i,0] # on récupère le prenom
    distances = distanceallp(prenom, poids1, poids2, poids3, poids4, poids5) # on calcule sa distance avec tous les autres
    distancesk = distances.head(k) # on affiche les k premiers
    kAlive = 0

    # On récupère la vraie valeur de isAlive
    alive = got.loc[got['name'] == prenom,'isAlive'] # on cherche le 1er prénom

    # On récupère les valeurs 'isAlive' des k plus proches voisins
    for j in range(k):
      prenomn = distancesk.iloc[j,0] # on récupère le prénom du k plus proche voisin
      personne = got.loc[got['name'] == prenomn, ('isAlive')].values
      kAlive += personne[0]
    mid = math.ceil(k/2)
    
    # On classifie selon les valeurs 'isAlive' des k plus proches voisins
    if kAlive >= mid:
      if alive.values[0] == 1:
        suc += 1
    else:
      if alive.values[0] == 0:
        suc += 1
  acc = suc / len(got)
  #acc = suc / 10
  print("Avec", k, "voisins et les poids suivants :", poids1,"pour la variable 'male'", poids2, "pour la variable 'book1'", poids3, "pour la variable 'book4'", poids4, "pour la variable 'numDeadRelations'", poids5, "pour la variable 'popularity', on obtient une précision de", acc)
  return poids1, poids2, poids3, poids4, poids4, k, acc

In [ ]:
predictpimp(param.iloc[0,0], param.iloc[0,1], param.iloc[0,2], param.iloc[0,3], param.iloc[0,4], param.iloc[0,5])